In [1]:
from tqdm import tqdm
import re 

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.document_loaders import DirectoryLoader

from src.utils import reformat_text, get_document_from_pkl
from src.vectorstore import create_collection, activate_collection

/Users/damir_bogdan/Library/Caches/pypoetry/virtualenvs/huberman-rag-09zSrCFX-py3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 30 files: 100%|██████████| 30/30 [00:00<00:00, 165782.77it/s]


In [2]:
create_collection()
collection = activate_collection()

[WARNING] PyMilvus: 


Collection already exists.


In [3]:
loader = DirectoryLoader("docs/youtube_test/", glob="*.txt", show_progress=True)
docs = loader.load()

for i, doc in enumerate(docs):
    doc.page_content = reformat_text(doc.page_content)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=100,
    length_function=len,
)

splitted_docs = text_splitter.split_documents(docs)

  0%|          | 0/1 [00:00<?, ?it/s]

In [3]:
documents = get_document_from_pkl("docs/document.pkl")

In [4]:
for i, doc in enumerate(documents):
    doc.page_content = reformat_text(doc.page_content)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=100,
    length_function=len,
)

splitted_docs = text_splitter.split_documents(documents)

In [ ]:
# from src.bgem3_embeddings import embed_bgem3
# for doc in tqdm(splitted_docs, desc="Processing and importing docs"):
#     doc = doc.page_content
#     sparse_embeddings, dense_embeddings = embed_bgem3(doc)
#     entity = [[doc], sparse_embeddings, dense_embeddings]
#     collection.insert(entity)
#     collection.flush()


In [ ]:
from src.bge_embeddigns import embed_bge
from src.splade_embeddings import embed_splade
for doc in tqdm(splitted_docs, desc="Processing and importing docs"):
    doc = doc.page_content
    dense_embeddings = embed_bge(doc)
    sparse_embeddings = embed_splade(doc)
    entity = [[doc], sparse_embeddings, [dense_embeddings]]
    collection.insert(entity)
    collection.flush()